## 参数管理
在选择了架构并设置了超参数后，我们就进⼊了训练阶段。此时，我们的⽬标是找到使损失函数最⼩化的模
型参数值。经过训练后，我们将需要使⽤这些参数来做出未来的预测。此外，有时我们希望提取参数，以便
在其他环境中复⽤它们，将模型保存下来，以便它可以在其他软件中执⾏，或者为了获得科学的理解⽽进⾏
检查。  

之前的介绍中，我们只依靠深度学习框架来完成训练的⼯作，⽽忽略了操作参数的具体细节。本节，我们将
介绍以下内容：
* 访问参数，⽤于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。


In [2]:
# 具有单隐藏层的多层感知机
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
print(X)
net(X)


tensor([[0.3267, 0.3245, 0.4916, 0.1650],
        [0.5152, 0.0704, 0.4901, 0.0600]])


tensor([[-0.0353],
        [-0.0197]], grad_fn=<AddmmBackward0>)

## 参数访问
从已有模型中访问参数。当通过Sequential类定义模型时，我们可以通过索引来访问模型的任意层。这
就像模型是⼀个列表⼀样，每层的参数都在其属性中。如下所⽰，我们可以检查第⼆个全连接层的参数。

In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1468,  0.0603,  0.1024, -0.1477, -0.2875,  0.2346, -0.0436, -0.1581]])), ('bias', tensor([0.1719]))])


输出的结果告诉我们⼀些重要的事情：⾸先，这个全连接层包含两个参数，分别是该层的权重和偏置。两者
都存储为单精度浮点数（float32）。注意，参数名称允许唯⼀标识每个参数，即使在包含数百个层的⽹络中也
是如此。

### 目标参数
注意，每个参数都表⽰为参数类的⼀个实例。要对参数执⾏任何操作，⾸先我们需要访问底层的数值。有⼏
种⽅法可以做到这⼀点。有些⽐较简单，⽽另⼀些则⽐较通⽤。下⾯的代码从第⼆个全连接层（即第三个神
经⽹络层）提取偏置，提取后返回的是⼀个参数类实例，并进⼀步访问该参数的值。


In [7]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)


<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1719], requires_grad=True)
tensor([0.1719])


参数是复合的对象，包含值、梯度和额外信息。这就是我们需要显式参数值的原因。除了值之外，我们还可
以访问每个参数的梯度。在上⾯这个⽹络中，由于我们还没有调⽤反向传播，所以参数的梯度处于初始状态。

In [8]:
net[2].weight.grad == None

True

### 一次性访问所有参数
当我们需要对所有参数执⾏操作时，逐个访问它们可能会很⿇烦。当我们处理更复杂的块（例如，嵌套块）
时，情况可能会变得特别复杂，因为我们需要递归整个树来提取每个⼦块的参数。下⾯，我们将通过演⽰来
⽐较访问第⼀个全连接层的参数和访问所有层。

In [11]:
# 访问第一个全连接层
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
# 访问所有层
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
#这为我们提供了另⼀种访问⽹络参数的⽅式
net.state_dict()['2.bias'].data


tensor([0.1719])

### 从嵌套块收集参数
如果我们将多个块相互嵌套，参数命名约定是如何⼯作的。我们⾸先定义⼀个⽣成块的函数（可
以说是“块⼯⼚”），然后将这些块组合到更⼤的块中。


In [13]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这⾥嵌套
        net.add_module(f'block {i}', block1())
    return net


rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)



tensor([[0.1980],
        [0.1980]], grad_fn=<AddmmBackward0>)

In [14]:
# 设计了⽹络后，我们看看它是如何⼯作的
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引⼀样访问它们。下⾯，我们访问第⼀个主要的块
中、第⼆个⼦块的第⼀层的偏置项。

In [15]:
rgnet[0][1][0].bias.data


tensor([-0.4689, -0.3548, -0.2061,  0.1927, -0.1766,  0.2679, -0.0772, -0.2289])

## 参数初始化
